In [1]:
# %load assignment2.py
import urllib.request
import base64

################################################################################
# 
# This starter file for UChicago CMSC 23200 / 33250 is for Python3
#
################################################################################

################################################################################
# 
# make_query(task, cnet_id, query)
# -- task should be one of 'one','two','three','four','five'
# -- cnet_id should always be your own cnet_id
# -- query can be any string of bytes, including non-printable
#
################################################################################

def make_query(task, cnet_id, query):
    DEBUG = False; # Replace with "True" to print extra debugging information
    task = task.lower()
    cnet_id = cnet_id.lower()
    if DEBUG: 
        print("Querying the server")
        print("(Task:", task, ")")
        print("(CNET ID:", cnet_id, ")")
        print("(Query:", query, ")")
    if (type(query) is bytearray) or (type(query) is bytes):
        url = "http://securityclass.cs.uchicago.edu/" + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnet_id) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query)) + "/"
    else:
        url = "http://securityclass.cs.uchicago.edu/" + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnet_id) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query.encode('utf-8'))) + "/"
    if DEBUG:
        print("(Querying:", url, ")")
    with urllib.request.urlopen(url) as response:
        answer = base64.urlsafe_b64decode(response.read())
        return answer

################################################################################
# Constants for the attacks - Don't change these!
################################################################################

e3 = 65537
k3 = 512
# Modulus N2 is 512 bits long, too short for real security
N3= 0x00dd9387a53d8eb960acc9d3bc49b859e9127ad571f95d3555dc5a30f08b832299d82ecbba38acdadfb4263947f86212f1a3894e3d308545f2618ec3a1cefc5bdf

e4 = 65537
k4 = 512
# Modulus N3 is 512 bits long, too short for real security
N4= 0x00c7d11981bf2838ed5ae602cecc4cffcf141537f9ec6e12b2fcaae43dedbf9845049066cc8720c6685d100957c07e4f5f97b2b8e66d1a3bcc32ecf1e0fee55e6f

e5 = 3
k5 = 2048
# Modulus N4 is 2048 bits long
N5 = 0x00bc9e8d81ce1de63e0ab302030e5c0595bf5d2c30fd2660ac9299431a29c4e231a675d684e35415ad87ca738509469aaa0455d62543ab9265d71767f55c7f5fdbb9e2618112212178417c21b4e8a98ab0980fd67864ed7e7e3dcefc3143d5e5d3be2bf0c36c75c977052fedbfdc1c2e448710338fad4fe0e3fa8fc2c662e3466d358df6618dc0a63f45395e5c5aa88d15a49ce2be791acbcd81e28533228918f6abb57e023145a97afea85ad238686f51409017a4d6af8687f7a9438f09a2d9d9e619abdde8e67fc95af23dc97b4a595baa26bfeaf16d31b93e3e1bae1f5813fcd9ef2c8f93df2dd4a779626d07852f120e6b84d936abb811fd4525d9a0cf6621


################################################################################
# Helper methods go below here
################################################################################

#
# modexp(base,exp,modulus) computes (base**exp) % modulus efficiently.
#
# In particular this method can handle very large values of exp, while
# the python builtin ** operator can not.
#
# Feel free to change this if you want, but you probably won't need to.
#
def modexp(base, exp, modulus):
    ret = 1
    while exp > 0:
        if exp % 2 == 1:
            ret = (ret*base) % modulus
        base = (base*base) % modulus
        exp = exp >> 1
    return ret

#your code here

In [2]:
import math

In [3]:
from decimal import *

In [4]:
def rsa_enc(N, e, M):
    return modexp(M, e, N)

def rsa_dec(N, d, C):
    return modexp(C, d, N)

In [6]:
def gen_ctext_given_msg_coef(ctext, coeff, exp, modulus):
    """Given C, creates C' that decodes to M' = coeff * M"""
    mult = modexp(coeff, exp, modulus)
    return ctext * mult % modulus

In [19]:
N0 = 10919004200013450018317318873934378737492160016069064494055844031850292055849078163405133164445025496900759761448073186739695639699762006862469647051348171
e0 = 65537
d0 = 3225202302575039526596985805905392428710090995789629529516655153097608276502165308334624255495652191210051352829959680248336718656470211430168810247400369

M = 3
C = rsa_enc(N0, e0, M)
rsa_dec(N0, d0, C), \
rsa_dec(N0, d0, gen_ctext_given_msg_coef(C, 2, e0, N0)) # produces 2M

(3, 6)

In [8]:
M = 107552524872943947383268605932222862319223730769362344735103445182683512969479
rsa_dec(N0, d0, rsa_enc(N0, e0, M))

107552524872943947383268605932222862319223730769362344735103445182683512969479

In [ ]:
make_query('four', 'davidcash', '')

In [9]:
def oracle(ctext, N, d):
    return rsa_dec(N, d, ctext) % 2

In [ ]:
print(oracle(1, 15, 3), 15 / 2) # > N/2
print(oracle(8, 15, 3), 45 / 4) # < 3N/4
print(oracle(4, 15, 3), 75 / 8) # < 5N/8
print(oracle(2, 15, 3), 7 * 15 / 16) # < 7N/16
print(oracle(1, 15, 3), 17 * 15 / 32) # > 17N/32
print(oracle(8, 15, 3), 35 * 15 / 64) # < 35N/64

In [22]:
def binary_search_with_oracle(ctext, modulus, encrypt, decrypt): # C, N, e
    getcontext().prec = k4
    lo, hi = Decimal(0), Decimal(modulus)
    step = Decimal(lo + hi) / 2
    mult = modexp(2, encrypt, modulus)
    ctext = ctext * mult % modulus
    while lo < hi:
        resp = oracle(ctext, modulus, decrypt)
        if resp == 0:
            hi -= step
        else:
            lo += step
        ret = math.ceil(lo)
        if math.floor(hi) == ret:
            return ret
        ctext = ctext * mult % modulus
        step /= 2

In [17]:
binary_search_with_oracle(2, 15, 3, 3)

3.75
1.875
0.9375


8

In [24]:
M = 107552524872943947383268605932222862319223730769362344735103445182683512969479
C = rsa_enc(N0, e0, M)
binary_search_with_oracle(C, N0, e0, d0)

107552524872943947383268605932222862319223730769362344735103445182683512969479

In [35]:
hex(M)

'0xedc89263ce8e17573b6e896a8bf06b45ce65ae33c7fb79ccddbcd5bbed8e1907'

In [67]:
def problem4(cnetid):
    getcontext().prec = k4
    ctext = int(make_query('four', cnetid, ''), 16)
    lo, hi = Decimal(0), Decimal(N4)
    step = Decimal(lo + hi) / 2
    mult = modexp(2, e4, N4)
    ctext = ctext * mult % N4
    ret = None
    while lo < hi:
        resp = make_query('four', cnetid, hex(ctext))
        if resp == b'\x00':
            hi -= step
        else:
            lo += step
        ret = math.ceil(lo)
        if math.floor(hi) == ret:
            break
        ctext = ctext * mult % N4
        step /= 2
    if ret is None:
        raise
    print('Q4 FLAG:', bytes.fromhex(format(ret, 'x')))

In [68]:
problem4('davidcash')

Q4 FLAG: b'Any chance collision and I light up in the dark'


In [69]:
problem4('rohankumar')

Q4 FLAG: b"And when we'll funk we'll hear beats Don't drag"


In [62]:
res = 39339006824372597283136697946419976052270116010140928208646408184004313718562251599451134607351573568943423644007

In [63]:
bytes.fromhex(format(res, 'x'))

b"And when we'll funk we'll hear beats Don't drag"